In [1]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import json
import csv
import time
import psycopg2

In [2]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
# import undetected_chromedriver as uc
# driver = uc.Chrome(headless=True,use_subprocess=False)

In [3]:
class LinkedinScraper:
    def __init__(self):
        self.driver = driver
        self.wait = wait
        
    @staticmethod
    def login():
        LinkedinScraper.get_linkedin_site()
        with open("cookies.json", "r") as file:
            cookies = json.load(file)
            for cookie in cookies:
                driver.add_cookie(cookie)

        # Refresh the page to apply cookies and maintain the session
        driver.refresh()
        
    @staticmethod
    def save_cookies():
        LinkedinScraper.get_linkedin_site()
        cookies = driver.get_cookies()
        # Save cookies for later use (to a file or variable)
        print(cookies)
        with open('cookies.json', 'w') as file:
            json.dump(cookies, file)

    
    @staticmethod
    def get_linkedin_site():
        url = f"https://www.linkedin.com/"
        driver.get(url)
        wait = WebDriverWait(driver, 30)
        
    @staticmethod
    def button_locator(lablel):
        return wait.until(EC.presence_of_all_elements_located((By.XPATH,f"//button[@aria-label='{lablel}']")))[0]

    def search(self,query):
        input_box = driver.find_element(By.CLASS_NAME, "search-global-typeahead__input")
        
        # Clear the input box
        input_box.clear()

        # Send text to the input box
        input_box.send_keys(query)

        # Press the Enter key
        input_box.send_keys(Keys.RETURN)
    
    @staticmethod
    def send_invite_with_message(message,send_note=True):
        while(LinkedinScraper.is_there_next_page()):
            LinkedinScraper.scroll_to_top()
            send_button = wait.until(EC.presence_of_all_elements_located((By.XPATH,"//button[@class='artdeco-button artdeco-button--2 artdeco-button--secondary ember-view']")))
             
            time.sleep(1)
            print(f"there are total  {len(send_button)} people on page")
            # send invite to all the people on the page
            for button in send_button:
                send_button = button
                time.sleep(1.5)
                # identify whether the add note button is there or not
                # check current button is contains follow or connect
                print(f"printing the aria label {button.get_attribute('aria-label')}")
                aria_text = button.get_attribute('aria-label')
                if "Follow" in aria_text:
                    print('follow button, do not follow')
                    continue
                elif "Invite" in aria_text:
                    print(f'do the invite for {aria_text}')
                    send_button.click()
                    # @remove 
                    # closing right now for testing
                    # close_button = LinkedinScraper.button_locator('Dismiss')
                    # close_button.click()
                     
                    # add_without_note_button = wait.until(EC.presence_of_all_elements_located((By.XPATH,"//button[@aria-label='Send without a note']")))[0]
                    with_note_button = wait.until(EC.presence_of_all_elements_located((By.XPATH,"//button[@aria-label='Add a note']")))[0]
                    with_note_button.click()
                    add_note_text = wait.until(EC.presence_of_all_elements_located((By.XPATH,"//textarea[@id='custom-message']")))[0]
                    add_note_text.send_keys(message)
                    send_button_on_add_note = wait.until(EC.presence_of_all_elements_located((By.XPATH,"//button[@aria-label='Send invitation']")))[0]
                    send_button_on_add_note.click()
            # got to the next page since it has next page
            # LinkedinScraper.scroll_entire_page()
            #@remove
            next_page = LinkedinScraper.go_to_next_page()
        
            if next_page == False:
                break
                
        
        
    @staticmethod
    def go_to_next_page():
        if LinkedinScraper.is_there_next_page():
            try:
                # try finding the next button if not found then scroll to the bootom of the page
                next_button = wait.until(EC.presence_of_all_elements_located((By.XPATH,"//button[@aria-label='Next']")))[0]
            except:
                LinkedinScraper.scroll_entire_page()
            try:
                next_button = wait.until(EC.presence_of_all_elements_located((By.XPATH,"//button[@aria-label='Next']")))[0]
                # click the next button
                next_button.click()
                time.sleep(1)
                return True
            except:
                print("can not go to the next page")
        else:
            print("no next page")
            return False
    
    @staticmethod
    def is_there_next_page():
        try:
            # try finding the next button if not found then scroll to the bootom of the page
            next_button = wait.until(EC.presence_of_all_elements_located((By.XPATH,"//button[@aria-label='Next']")))[0]
           
        except:
            LinkedinScraper.scroll_entire_page()
        
        try:
            next_button = wait.until(EC.presence_of_all_elements_located((By.XPATH,"//button[@aria-label='Next']")))[0]
            is_disabled = next_button.get_attribute('disabled')
            if is_disabled:
                return False
            else:
                return True
        except:   
            return False
        
    @staticmethod
    def scroll_entire_page():
        # do smooth scroll to the bottom of the page
        # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.execute_script("window.scrollTo({ top: document.body.scrollHeight, behavior: 'smooth' });")
        # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # time.sleep(1)
        
    @staticmethod
    def scroll_to_top():
        driver.execute_script("window.scrollTo(0, 0);")
        
    @staticmethod
    def get_all_people(name):
        # get all the people in the search result
        driver.get(f'https://www.linkedin.com/search/results/people/?keywords={name}&origin=CLUSTER_EXPANSION&sid=b%2CA')

        
            

In [4]:
scraper = LinkedinScraper()

In [5]:
LinkedinScraper.login()

In [6]:
LinkedinScraper.get_all_people('eric')

### final Script

In [8]:
LinkedinScraper.send_invite_with_message("hello I want to connect with you")

there are total  10 people on page
printing the aria label Invite Eric Kang to connect
do the invite for Invite Eric Kang to connect


TimeoutException: Message: 
